In [2]:
import pandas as pd
from pathlib import Path
X_train = pd.read_csv(Path(r"C:\Users\90553\Documents\ml_data_analysis\data\X_train.csv"),sep=r"\s+")
Y_train = pd.read_csv(Path(r"C:\Users\90553\Documents\ml_data_analysis\data\Y_train.csv"),header=None, names=['category'])
X_test = pd.read_csv(Path(r"C:\Users\90553\Documents\ml_data_analysis\data\X_test.csv"),sep=r"\s+")
Y_test = pd.read_csv(Path(r"C:\Users\90553\Documents\ml_data_analysis\data\Y_test.csv"),header=None, names=['category'])
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(1668, 9635) (1668, 1) (557, 9635) (557, 1)


In [3]:
XY_train = pd.concat([Y_train, X_train], axis=1)
print(XY_train.head())

   category  ad  sale  boost  time  warner  profit  quarterli  media  giant  \
0         4   1     1      0     0       0       0          0      4      1   
1         1   0     0      0     0       0       0          0      0      0   
2         1   1     0      0     0       0       0          0      0      0   
3         0   0     0      1     0       0       0          0      0      0   
4         0   0     0      0     1       0       0          0      0      0   

   ...  Â£339  denialofservic  ddo  seagrav  bot  wirelessli  streamcast  \
0  ...      0               0    0        0    0           0           0   
1  ...      0               0    0        0    0           0           0   
2  ...      0               0    0        0    0           0           0   
3  ...      0               0    0        0    0           0           0   
4  ...      0               0    0        0    0           0           0   

   peripher  headphon  flavour  
0         0         0        0  
1 

In [4]:
word_freq_sum = XY_train.groupby('category').sum()
print(word_freq_sum.head())



           ad  sale  boost  time  warner  profit  quarterli  media  giant  \
category                                                                    
0         125   284     87   164       0     187          9     23     69   
1          88    95     12   159      10       7          0     31     12   
2         142     8     34   239       0       3          1     27      3   
3         157    34     16   262       0       0          0     19      5   
4         119    91     17   284      10      18          2    145     45   

          jump  ...  Â£339  denialofservic  ddo  seagrav  bot  wirelessli  \
category        ...                                                         
0           28  ...      0               0    0        0    0           0   
1            4  ...      0               0    0        0    0           0   
2            3  ...      0               0    0        0    0           0   
3           40  ...      0               0    0        0    0           0  

In [5]:
total_words_per_category = word_freq_sum.sum(axis=1)
print(total_words_per_category)

category
0    63889
1    48010
2    71554
3    62439
4    78433
dtype: int64


In [28]:
import numpy as np
word_probs = word_freq_sum.div(total_words_per_category, axis=0)

# take log normally (this will give -inf where prob=0)
log_probs = np.log(word_probs)

# replace -inf with a very negative number to simulate it
#log_probs = log_probs.replace(-np.inf, -1e12)


print(log_probs.head())
print(log_probs.shape)
print(word_probs.sum(axis=1))

                ad      sale     boost      time    warner     profit  \
category                                                                
0        -6.236589 -5.415928 -6.598994 -5.965036      -inf  -5.833794   
1        -6.301828 -6.225288 -8.294258 -5.710260 -8.476580  -8.833254   
2        -6.222381 -9.098766 -7.651847 -5.701744      -inf -10.079595   
3        -5.985700 -7.515585 -8.269357 -5.473601      -inf       -inf   
4        -6.490877 -6.759141 -8.436787 -5.621026 -8.967415  -8.379628   

          quarterli     media      giant       jump  ...      Â£339  \
category                                             ...              
0         -8.867678 -7.929408  -6.830796  -7.732698  ...       -inf   
1              -inf -7.345177  -8.294258  -9.392870  ...       -inf   
2        -11.178208 -7.882371 -10.079595 -10.079595  ...       -inf   
3              -inf -8.097506  -9.432507  -7.353066  ...       -inf   
4        -10.576853 -6.293266  -7.463338  -9.660562  ... -10.5

c:\Users\90553\Documents\ml_data_analysis\venv\Lib\site-packages\pandas\core\internals\blocks.py:395: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [29]:
X = X_test.to_numpy() #mxn matrix of test data
T = log_probs.to_numpy() #kxn matrix of log prob of words given class
print(X.shape)
print(T.shape)


(557, 9635)
(5, 9635)


In [30]:
scores = X @ T.T              # (n_docs × n_classes)

print(scores.shape)           # should be (num_docs, num_categories)
print(scores)

(557, 5)
[[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 ...
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]


C:\Users\90553\AppData\Local\Temp\ipykernel_4220\3890831514.py:1: RuntimeWarning: invalid value encountered in matmul
  scores = X @ T.T              # (n_docs × n_classes)


In [31]:
priors = Y_train['category'].value_counts().sort_index() / len(Y_train)
print(priors)
print(priors.sum())
priors = np.log(priors)

priors = priors.reset_index()
priors.columns = ['category', 'probability']
priors_array = priors['probability'].to_numpy()
print(priors_array)

category
0    0.224820
1    0.170264
2    0.191247
3    0.226019
4    0.187650
Name: count, dtype: float64
1.0
[-1.49245456 -1.77040634 -1.65418948 -1.4871354  -1.67317739]


In [43]:
final_scores = scores + priors_array
print(final_scores.shape)
print(final_scores)


(557, 5)
[[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 ...
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]


In [49]:
import numpy as np

# Assume: scores (N×K) and Y_test (N×1) are numpy arrays
# Replace NaNs (e.g. from -inf + something) with -inf
final_scores = np.nan_to_num(final_scores, nan=-np.inf)

# 1. Predicted class indices
# np.argmax already picks the smallest index when values are equal
y_pred = np.argmax(final_scores, axis=1)

# 2. True labels
y_true = Y_test.to_numpy().flatten()   # or np.array(Y_test).flatten() if already numeric

# 3. Accuracy
accuracy = np.mean(y_true == y_pred)
print(f"Accuracy: {accuracy:.3f}")

# 4. Confusion matrix (NumPy only)
K = len(np.unique(y_true))
conf_matrix = np.zeros((K, K), dtype=int)
for t, p in zip(y_true, y_pred):
    conf_matrix[t, p] += 1

print("Confusion matrix")
print(conf_matrix)



Accuracy: 0.242
Confusion matrix
[[135   0   0   0   0]
 [102   0   0   0   0]
 [ 98   0   0   0   0]
 [134   0   0   0   0]
 [ 88   0   0   0   0]]
